In [167]:
import pandas as pd 
from scipy import stats   
import numpy as np 
import matplotlib.pyplot as plt
from plotting import master_plot
from tabulate import tabulate

In [190]:
def get_good_df(inputs,str_inputs,path,case='general'):


    df=pd.read_csv(path,usecols=inputs)
    print('\n Original shape = ',df.shape)
    #file.write('A look at the OLD df: {}'.format(df.info(verbose=True)))
    file.write('\n Statistical properties of file (before): \n {0} \n'.format(df.describe()))


    if case=='general':
        # only for numerical dfs
        df.replace([np.inf, -np.inf], np.nan)
        df.dropna(inplace=True)
        df = df[(df[inputs] != 0).all(axis=1)]
        print('Shape of df after invalid removal =', df.shape)
        z_scores=np.abs(stats.zscore(df))
        new_df=df[(z_scores < 3.0).all(axis=1)]
        df=new_df
        print('Shape of new df',df.shape)
    

    else:
        #remove all 0, inf, and NaN and store index from numerical df
        invalid_index=np.array([])
        for column in inputs:
            result=df[(df[column].isnull())| (df[column]== 0)| (df[column]== np.inf)].index.tolist()
            invalid_index=np.append(invalid_index,result)
        invalid_index=np.unique(invalid_index)
        df.drop(df.index[invalid_index.astype(int)],inplace=True)
        print('Shape of df after invalid removal =', df.shape)

        # massage string values
        df_str=pd.read_csv(path,usecols=str_inputs)
        print('length of original titles = ',len(df_str))
        df_str.drop(df_str.index[invalid_index.astype(int)],inplace=True)
        print('length of original titles after zero index drop = ',len(df_str))
        
        #combine both string and num df
        df=pd.concat([df,df_str],axis=1)

        # delete rows in all cols
        table=pd.DataFrame(columns=['column','z-score','movie','value'])
        
        for column in inputs:
            print('column =',column)
            z_scores=np.abs(stats.zscore(df[column].astype('float')))
            index_names = np.where(z_scores > 3.0)[0] # creates a nested array (index=mxn where m=1=list)
            print('length of index names =',len(index_names))
            for index in index_names:
                table = table.append({'column' : column , 'z-score' : z_scores[index],'movie':df.loc[index,'title'],'value':df.loc[index,column]} , ignore_index=True)
                #file.write('\n {0} outlier (z-score = {1:.2f}), row = {2}, value = {3}'.format(column,z_scores[index],df.loc[index,'title'],df.loc[index,column]))
            print('Number of outliers = ',len(index_names))
            df.drop(df.index[index_names],inplace=True)
            df.reset_index(drop=True,inplace=True)
            print('Length of first column (should change after each iteration) = ',len(df[inputs[0]]))


        # return new df
        file.write(tabulate(table, headers='keys',tablefmt="grid"))
        file.write('\n \n Statistical properties of data (after): \n {0} '.format(df.describe()))
        print('Final shape = ',df.shape)

        
    # aspect ratio adjustment
    if 'aspect ratio' in df.columns:
        df=change_ar(df)
        
    df.to_csv(csv_path,index=False)
    return df

In [191]:
def get_ROI(df):
    '''gets the ROI for all movies'''

    df['ROI']=((df['Cumulative Worldwide Gross']-df['Budget'])/df['Budget'])*100
    df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df

In [192]:
def change_ar(df):
    '''changes the aspect ratio into something a little more readable'''

    df['aspect ratio'] = [df.loc[s,'aspect ratio'].split(':',1)[0] for s in range(len(df))]
    return df

In [193]:
def popularity(df):
    '''calulates popularity model for each movie'''

In [194]:
if __name__=='__main__':

    path = '/Users/austinbenny/Documents/python/movie_budget_ratings/IMDB_data/imdb_data_final_v2.csv'
    text_path = '/Users/austinbenny/Documents/python/movie_budget_ratings/output/output.txt'
    csv_path='/Users/austinbenny/Documents/python/movie_budget_ratings/output/newDataframe.csv'


    file = open(text_path,'w+')


    inputs=['Budget','rating','Cumulative Worldwide Gross','Opening Weekend United States','runtimes','votes']
    str_inputs=['title','aspect ratio']

    df=get_good_df(inputs,str_inputs,path,case='not_general')


 Original shape =  (2067, 6)
Shape of df after invalid removal = (2067, 6)
length of original titles =  2067
length of original titles after zero index drop =  2067


TypeError: Index(...) must be called with a collection of some kind, False was passed

In [165]:
df=get_ROI(df)
df.sort_values('ROI',ascending=False)

,Cumulative Worldwide Gross,Budget,Opening Weekend United States,rating,runtimes,votes,title,aspect ratio,ROI
235,123606280.0,20.0,6896612.0,7.3,114,135125.0,Poltergeist (1982),2.20,6.180313e+08
1208,193355800.0,15000.0,19617650.0,6.3,86,213655.0,Paranormal Activity (2007),1.85,1.288939e+06
332,248639099.0,60000.0,29207381.0,6.5,81,223362.0,The Blair Witch Project (1999),1.33,4.142985e+05
816,54667954.0,120000.0,1100943.0,5.7,79,49486.0,Open Water (2003),1.85,4.545663e+04
99,30000000.0,114000.0,5452.0,7.9,96,110241.0,Night of the Living Dead (1968),1.37,2.621579e+04
...,...,...,...,...,...,...,...,...,...
599,2683893.0,100000000.0,2182900.0,3.8,95,23757.0,The Adventures of Pluto Nash (2002),1.85,-9.731611e+01
926,222340.0,10000000.0,80602.0,7.3,120,25656.0,Peaceful Warrior (2006),2.35,-9.777660e+01
186,612335.0,48000000.0,7403362.0,6.3,122,15594.0,Red Corner (1997),1.85,-9.872430e+01
1590,109099.0,10000000.0,239034.0,5.8,99,7171.0,High School (2010),2.35,-9.890901e+01


In [171]:
# explore tabulate
table = [["Sun",696000,1989100000],["Earth",6371,5973.6],["Moon",1737,73.5],["Mars",3390,641.85]]
print(tabulate(table,tablefmt="grid"))

+-------+--------+---------------+
| Sun   | 696000 |    1.9891e+09 |
+-------+--------+---------------+
| Earth |   6371 | 5973.6        |
+-------+--------+---------------+
| Moon  |   1737 |   73.5        |
+-------+--------+---------------+
| Mars  |   3390 |  641.85       |
+-------+--------+---------------+


In [173]:
table=pd.DataFrame(columns=['column','z-score','movie','value'])
#table = table.append({'column' : column , 'z-score' : z_scores[index],'movie':df.loc[index,'title'],'value':df.loc[index,column]} , ignore_index=True)

In [174]:
table

,column,z-score,movie,value
